<a href="https://colab.research.google.com/github/karimabbes/note_taker/blob/main/transcriber_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn pyngrok nest-asyncio python-multipart
!pip install git+https://github.com/zackees/transcribe-anything.git


  Cloning https://github.com/zackees/transcribe-anything.git to /tmp/pip-req-build-l_gisibc
  Running command git clone --filter=blob:none --quiet https://github.com/zackees/transcribe-anything.git /tmp/pip-req-build-l_gisibc
  Resolved https://github.com/zackees/transcribe-anything.git to commit 9281bdea3a38ddc93e1b3c002ab53e2dae1125b6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# ⚙️ Step 2: Import modules and set up FastAPI + ngrok
import nest_asyncio
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from pyngrok import ngrok, conf
import uvicorn
import tempfile
import os
import transcribe_anything as ta
import nest_asyncio

conf.get_default().auth_token = "2ZWnfh1fWxmNVe6duA5erHAj9AA_2FXEESgv7sswWM4V2vvKN"  # paste your token here

app = FastAPI()
nest_asyncio.apply()

# Set up ngrok tunnel
public_url = ngrok.connect(8000)
print(f"🚀 Public API URL: {public_url}/transcribe")

nest_asyncio.apply()


🚀 Public API URL: NgrokTunnel: "https://bfd0-34-82-83-235.ngrok-free.app" -> "http://localhost:8000"/transcribe


In [3]:
# 📥 Step 3: Define transcription endpoint
@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    try:
        # Save the uploaded file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp:
            tmp.write(await file.read())
            tmp_path = tmp.name

        # Use transcribe-anything
        transcript = ta.transcribe_anything(url_or_file=tmp_path, output_dir="output_dir", task="transcribe", model="large", device="cuda")
        os.remove(tmp_path)
        return JSONResponse(content={"transcript": transcript})
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

In [5]:
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [9277]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     109.7.252.95:0 - "GET /apple-touch-icon.png HTTP/1.1" 404 Not Found
INFO:     109.7.252.95:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     109.7.252.95:0 - "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 Not Found

Download of https://github.com/zackees/ffmpeg_bins/raw/main/v5.0/linux.zip -> /usr/local/lib/python3.11/dist-packages/static_ffmpeg/bin/linux.zip completed.
Extracting /usr/local/lib/python3.11/dist-packages/static_ffmpeg/bin/linux.zip -> /usr/local/lib/python3.11/dist-packages/static_ffmpeg/bin
making dir output_dir


Running:
  /usr/local/bin/static_ffmpeg -y -i /tmp/tmpt1c76c7g.mp3 -acodec pcm_s16le -ar 44100 -ac 1 out.wav


#####################################
######### GPU ACCELERATED! ##########
#####################################
Using device cuda
Running whisper on /tmp/tmpiftp6wux.wav (will install models on first run)
Purging...
venv_path does not exist.


Debug: Detected NVIDIA availability: True for fingerprint: Linux-x86_64-#1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025-nvidia_smi:True
Running:
  whisper /tmp/tmpiftp6wux.wav --device cuda --model large --output_dir /tmp/tmphh0jbr4j --task transcribe


Done! Files were saved to /content/output_dir


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9277]
